In [8]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

## Possible Preproccessing

In [10]:
# fix bad columns
housing['date'] = [x.split('T')[0] for x in housing['date']]
housing['opinion_score'] = housing['view'] + housing['condition'] + housing['grade']
housing['year_of_data'] = [int(x[0:4]) for x in housing['date']]
housing['years_since_renovation'] = housing['year_of_data'] - (housing['yr_renovated'].where(housing['yr_renovated'] != 0,housing['yr_built']))


# For each feature normalize values between 0 and 1
normalized_housing = pd.DataFrame()
to_normalize = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
        'floors', 'view', 'condition', 'grade',
        'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15', 'opinion_score', 'years_since_renovation']

scaler = MinMaxScaler()

for x in housing.columns:
  if x in to_normalize:
    normalized_housing[x] = scaler.fit_transform(housing[ [x] ])
  else:
    normalized_housing[x] = housing[x]